In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
stations=pd.read_csv('charging_stations.csv')
station_city=stations['City']
stations.head()

#these are our charging station points. Each row represents one charging station in the appropriate city.
#our problem to fix: turn city into county. Then make a new df with the counts of charging stations by county

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Connector Types,Country,Intersection Directions (French),Access Days Time (French),BD Blends (French),Groups With Access Code (French),Hydrogen Is Retail,Access Code,Access Detail Code,Federal Agency Code
0,ELEC,LADWP - Truesdale Center,11791 Truesdale St,NaN,Sun Valley,CA,91352,NaN,NaN,E,...,J1772,US,NaN,NaN,NaN,Privé - Réservé au gouvernement,NaN,private,GOVERNMENT,NaN
1,ELEC,Southern California Edison - Rosemead Office B...,2244 Walnut Grove Ave,NaN,Rosemead,CA,91770,NaN,NaN,E,...,J1772 NEMA520,US,NaN,NaN,NaN,Privé,NaN,private,NaN,NaN
2,ELEC,Los Angeles Convention Center,1201 S Figueroa St,West hall,Los Angeles,CA,90015,NaN,213-741-1151,E,...,J1772 NEMA515,US,NaN,NaN,NaN,Public,NaN,public,NaN,NaN
3,ELEC,LADWP - John Ferraro Building,111 N Hope St,Across Hope,Los Angeles,CA,90012,NaN,800-473-3652,E,...,CHADEMO J1772 J1772COMBO,US,NaN,NaN,NaN,Public,NaN,public,NaN,NaN
4,ELEC,Cherokee & Hollywood Parking Garage,1718 N Cherokee Ave,NaN,Los Angeles,CA,90028,NaN,213-550-9904,E,...,J1772,US,NaN,NaN,NaN,Public,NaN,public,NaN,NaN


In [74]:
cnty=pd.read_csv('county_city.csv').drop(['Type','Land area[1]',
                                          'Incorporated[7]',
                                          'Population (2010)[1][8][9]',
                                         'Land area[1].1'],axis=1)

cnty.head()

#this has the link between city ane county

,Name,County
0,Name,County
1,Adelanto,San Bernardino
2,Agoura Hills,Los Angeles
3,Alameda,Alameda
4,Albany,Alameda


In [75]:
station_city[4] #well be interating over this to get a county label from this printed city/town label

'Los Angeles'

In [46]:
new_cities=[]
for city in station_city:
    new_cities.append((city.title().strip(',')))
#Some values are not capitlized or have commas. This undues human error

In [76]:
len(new_cities) #total amount of charger stations

5392

In [79]:
county_names=[] #end result is a list of the associated county names


for i in range(len(new_cities)):
    city=stations['City'][i]  #select the city string
    
    
    result=cnty[ cnty['Name'] == city] #link it to the dataframe where we know cities to counties
    
    
    if result.shape[0]==1: #if we get a result, get the county label and append it 
        county_found=np.array(result['County'])[0]

        county_names.append(county_found)
        
        
    else: #if we dont get a result (the city input has no county because it was mispelled or not written correctly,
                                    # lets append a 0 to the county name so we can keep track of the lost cities)
        county_names.append(0) 
        
county_names[:5] 

[0, 'Los Angeles', 'Los Angeles', 'Los Angeles', 'Los Angeles']

In [50]:
charging_stations_filtered=pd.DataFrame()
charging_stations_filtered['County']=county_names
charging_stations_filtered['City']=stations['City']
charging_stations_filtered.head(5)

,County,City
0,0,Sun Valley
1,Los Angeles,Rosemead
2,Los Angeles,Los Angeles
3,Los Angeles,Los Angeles
4,Los Angeles,Los Angeles


In [58]:
#lets make a count for how many charging stations each county has

stations_by_county=charging_stations_filtered.groupby('County').aggregate(len)  #len does the same as count
display(stations_by_county.head())
print(stations_by_county.shape)

#our data says it detected a nonzero amount of charging stations for 49 counties, which means we have 9 counties that
#essentially are 0. In addition, we lost 10% of our data due to mislabeled cities.

,City
County,
0,469
Alameda,262
Amador,10
Butte,14
Colusa,1


(50, 1)


In [80]:
lost_data=charging_stations_filtered[charging_stations_filtered['County']==0]
lost_data.dropna().head()

#for viewing purposes to see which cities were lost. Some were written as S.F. others are neighborhoods, others are 
#names of colleges.

,County,City
0,0,Sun Valley
8,0,Wilmington
9,0,Sylmar
16,0,La Jolla
19,0,Sepulveda


In [57]:
#stations_by_county.to_csv('/Users/nathancarrizales-duarte/Desktop/Grid/EV_model_all/charging_stations_edit.csv')